# Labelme to Yolo

In [6]:
# import json
  
# # Opening JSON file
# f = open('BKF-BLUE-1_phone_ (150).json')
  
# # returns JSON object as 
# # a dictionary
# data = json.load(f)
  
# # Iterating through the json
# # list
# for i in data['shapes']:
#     print(i)
  
# # Closing file
# f.close()

In [8]:
import json
import os
import cv2
from tqdm import tqdm

def labelme_to_yolo(json_path, class_map, image_folder, file_name):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # image_width = data['imageWidth']
    # image_height = data['imageHeight']

    image = cv2.imread(image_folder + file_name+ '.jpg')
    image_height, image_width, channels = image.shape

    yolo_lines = []
    for shape in data['shapes']:
        
        label = shape['label']
        class_index = class_map.get(label, -1)
        if class_index == -1:
            print("Class map out of dict, Please edit class map")
            continue
            
        points = shape['points']
        x_min = min(points, key=lambda x: x[0])[0]
        x_max = max(points, key=lambda x: x[0])[0]
        y_min = min(points, key=lambda x: x[1])[1]
        y_max = max(points, key=lambda x: x[1])[1]


        x_center = (x_min + x_max) / (2.0 * image_width)
        y_center = (y_min + y_max) / (2.0 * image_height)
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_lines.append(f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_lines

# Example usage:
if __name__ == '__main__':
    focus_site = "073"
    # focus_folder = './batch1-2/person-phone/crop_phone'
    focus_folder = f'../assets/crop-img/{focus_site}/images/'
    # focus_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/images/'
    # focus_folder = './img/'
    file_list = os.listdir(focus_folder)
    json_files = [filename for filename in file_list if filename.endswith('.json')]


    for json_file in tqdm(json_files):
        # json_file = 'BKF-BLUE-1_phone_ (150).json'
        source_dir = os.path.join(focus_folder, json_file)
        # output_dir = './yolo_label'
        output_dir = f"../assets/crop-img/{focus_site}/labels_coco/"
        class_map = {
            # 'car' : 0
            'person': 0,
            'phone': 67,
            # Add more class mappings as needed
        }

        name = json_file.split('.json')[0]
        os.makedirs(output_dir, exist_ok=True)

        yolo_lines = labelme_to_yolo(source_dir, class_map, image_folder= focus_folder, file_name=name)
        image_name = os.path.splitext(os.path.basename(json_file))[0]
        output_file = os.path.join(output_dir, f"{image_name}.txt")

        with open(output_file, 'w') as f:
            f.write('\n'.join(yolo_lines))

  0%|          | 0/102 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'shape'

In [1]:
# file_list = os.listdir('batch1-2')

# json_files = [filename for filename in file_list if filename.endswith('.json')]

# print(json_files)

In [4]:
import json
  
# Opening JSON file
f = open('BKF-BLUE-1_phone_ (150).json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['shapes']:
    print(i)
  
# Closing file
f.close()

{'label': 'person_with_phone', 'points': [[1608.108108108108, 1105.4054054054054], [2132.4324324324325, 1505.4054054054054]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'person', 'points': [[2278.3783783783783, 1086.4864864864865], [2513.5135135135133, 1232.4324324324325]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}


In [8]:
import json
import os
def labelme_to_yolo(labelme_annotation, image_width, image_height, class_map=None):
    label = labelme_annotation['label']
    points = labelme_annotation['points']

    x_min = min(points, key=lambda x: x[0])[0]
    x_max = max(points, key=lambda x: x[0])[0]
    y_min = min(points, key=lambda x: x[1])[1]
    y_max = max(points, key=lambda x: x[1])[1]

    x_center = (x_min + x_max) / (2.0 * image_width)
    y_center = (y_min + y_max) / (2.0 * image_height)
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height

    yolo_format = f"{class_map[label] if class_map else label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    return yolo_format

In [9]:
# Sample labelme annotation data
labelme_annotation = {
    'label': 'person_with_phone',
    'points': [[1608.108108108108, 1105.4054054054054], [2132.4324324324325, 1505.4054054054054]],
}

# Image width and height (you need to set these values according to your image size)
image_width = 2600
image_height = 2600

# Class index mapping (optional)
class_map = {
    'person': 0,
    'person_with_phone': 1,
    # Add more class mappings as needed
}

yolo_annotation = labelme_to_yolo(labelme_annotation, image_width, image_height, class_map)
print(yolo_annotation)

1 0.719335 0.502079 0.201663 0.153846


# Convert yolo to labelme

In [1]:
import os
import json
import cv2
from tqdm import tqdm

label_files = os.listdir('./person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/labels/')
label_files[:5]

['BKF-Inspection_phone_2023-12-06_09-37-07.436000.txt',
 'BKF-Inspection_phone_2023-12-28_08-57-11.100000.txt',
 'BKF-Inspection_phone_2023-12-28_08-19-22.174000.txt',
 'BKF-Inspection_phone_2023-12-28_08-02-17.500000.txt',
 'BKF-Inspection_phone_2023-12-28_07-58-23.063000.txt']

In [2]:
class_map = {
            0.0: 'person',
            1.0: 'phone',
            # Add more class mappings as needed
        }

for file_txt_name in tqdm(label_files):
    file_name = file_txt_name.split('.txt')[0]
    image_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/images/'
    txt_folder = './person-phone/post-prd-12-2023/post-prd-batch-12-2023/phone/labels/'

    image = cv2.imread(image_folder + file_name+ '.jpg')
    image_height, image_width, channels = image.shape

    
    yolo_labels = open(txt_folder + file_name + '.txt', "r").readlines()

    labelme_annotations = []
    # For each YOLO label
    for yolo_label in yolo_labels:
        values = yolo_label.split()
        class_id, x_center, y_center, width, height = map(float, values)
        # Calculate LabelMe coordinates
        x_top_left = int((x_center - width / 2) * image_width)
        y_top_left = int((y_center - height / 2) * image_height)
        x_bottom_right = int((x_center + width / 2) * image_width)
        y_bottom_right = int((y_center + height / 2) * image_height)
        
        labelme_annotation = {
            "label": class_map[class_id],  # Replace with actual class name
            "points": [[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]],
            "description": "",
            "shape_type": "rectangle"
        }
        
        # Create or append to the LabelMe JSON
        # You'll need to manage annotations per image in your actual code
        labelme_annotations.append(labelme_annotation)

    json_file_full = {
        "version": "5.3.0a0",
        "flags": {},
        "shapes" : labelme_annotations,
        "imagePath": file_name + '.jpg',
        "imageData" : None
    }


    with open("./img/" + file_name + ".json", "w") as json_file:
        json.dump(json_file_full, json_file)


  0%|          | 0/1342 [00:00<?, ?it/s]

100%|██████████| 1342/1342 [00:22<00:00, 58.55it/s]


In [31]:
# import json

# # Read YOLO labels
# yolo_labels = open("./yolo_label/BKF-BLUE-1_phone_%20(55).txt", "r").readlines()

In [36]:
# import base64


# image_width = 2600
# image_height = 2600


# class_map = {
#             0.0: 'person',
#             1.0: 'phone',
#             # Add more class mappings as needed
#         }
# labelme_annotations = []

# # For each YOLO label
# for yolo_label in yolo_labels:
#     values = yolo_label.split()
#     class_id, x_center, y_center, width, height = map(float, values)
#     # Calculate LabelMe coordinates
#     x_top_left = int((x_center - width / 2) * image_width)
#     y_top_left = int((y_center - height / 2) * image_height)
#     x_bottom_right = int((x_center + width / 2) * image_width)
#     y_bottom_right = int((y_center + height / 2) * image_height)
    
#     labelme_annotation = {
#         "label": class_map[class_id],  # Replace with actual class name
#         "points": [[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]],
#         "description": "",
#         "shape_type": "rectangle"
#     }
    
#     # Create or append to the LabelMe JSON
#     # You'll need to manage annotations per image in your actual code
#     labelme_annotations.append(labelme_annotation)

# json_file_full = {
#     "version": "5.3.0a0",
#     "flags": {},
#     "shapes" : labelme_annotations,
#     "imagePath": "BKF-BLUE-1_phone_%20(55).jpg",
#     "imageData" : None
# }


# with open("./yolo_label/BKF-BLUE-1_phone_%20(55).json", "w") as json_file:
#     json.dump(json_file_full, json_file)

In [33]:
labelme_annotations

[{'label': 'phone',
  'points': [[891, 644], [973, 734]],
  'description': '',
  'shape_type': 'rectangle'},
 {'label': 'person',
  'points': [[797, 666], [1183, 1131]],
  'description': '',
  'shape_type': 'rectangle'}]

# convert yolo to coco class

In [13]:
import os
import json
import cv2
from tqdm import tqdm

In [25]:
# # Define the file path
# file_path = '../assets/crop-img/073/labels/0730_14.txt'

# # Read the file
# with open(file_path, 'r') as file:
#     lines = file.readlines()
#     print(f"old : {lines}")

# # Process the lines
# new_lines = []
# for line in lines:
#     if line.startswith('1 '):
#         new_line = line.replace('1 ', '67 ', 1)
#         new_lines.append(new_line)
#     else:
#         new_lines.append(line)
# print(f"replace to : {new_lines}")

In [30]:
focus_site = "NKA"
input_dir = f'../assets/crop-img/{focus_site}/labels'
output_dir = f'../assets/crop-img/{focus_site}/labels_coco'


label_files = os.listdir(input_dir)
os.makedirs(output_dir, exist_ok=True)
label_files[:5]

['NKA-ch8_1.txt',
 'NKA-ch8_2.txt',
 'NKA-ch8_3.txt',
 'NKA-ch8_4.txt',
 'NKA-ch8_6.txt']

In [31]:
for file_name in tqdm(label_files):
    # Read the input file
    # print(os.path.join(input_dir, file_name))
    # print(input_dir + '/' + file_name)
    with open(os.path.join(input_dir, file_name), 'r') as file:
        lines = file.readlines()
        # print(lines)

    # Process the lines
    new_lines = []
    for line in lines:
        if line.startswith('1 '):
            new_line = line.replace('1 ', '67 ', 1) # change phone to cell phone coco
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Write the updated content to the new file
    with open(os.path.join(output_dir,file_name), 'w') as file:
        file.writelines(new_lines)

100%|██████████| 33/33 [00:00<00:00, 552.32it/s]
